In [ ]:
!pip install category_encoders
!pip install pdpbox
!pip install shap
!pip install eli5

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.rc('font', family='NanumBarunGothic') 
mpl.rc('axes', unicode_minus=False)

from category_encoders import OneHotEncoder, OrdinalEncoder, TargetEncoder

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, validation_curve, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, plot_confusion_matrix, classification_report, roc_curve
from sklearn.metrics import roc_auc_score, fbeta_score, recall_score, precision_score, confusion_matrix, r2_score

from scipy.stats import randint, uniform

from xgboost import XGBRegressor, XGBClassifier
from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
import shap
from eli5.sklearn import PermutationImportance
import eli5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
job_clean = pd.read_csv('/content/drive/My Drive/data_job_group.csv')
job_clean

### ※ 기준모델 설명 Rating 4.0 이상 만족 & 추천


#### 기준모델 분포 확인 및 평가지표

In [ ]:
# 4.0 기준으로 1(만족), 0 구분
print(job_clean[job_clean["Rating"] >= 4.0].shape[0] / job_clean.shape[0]) # 약 35%

# 분포 확인
sns.displot(job_clean["Rating"], kde=True);
plt.axvline(4.0, color='red');

In [ ]:
job_clean["Recommend"] = job_clean["Rating"] >= 4.0
job_clean["Recommend"] = job_clean["Recommend"].astype(int)
job_clean.head(3)

In [ ]:
job_clean.head(3).T

In [ ]:
job_clean["Recommend"].value_counts()

In [ ]:
job_clean["Recommend"].value_counts(normalize=True)

In [ ]:
# baseline model
major = job_clean["Recommend"].mode()[0]
job_pred = [major] * len(job_clean["Recommend"])
print("Baseline model accuracy:", accuracy_score(job_clean["Recommend"], job_pred))
print("Baseline model f1_score :", f1_score(job_pred, job_clean["Recommend"]))

## ※ baseline model : 0.649

### ※ 데이터 누수 제거


*   target이 될 Recommend의 특성과 동일한 Rating을 삭제 함으로써 분류모델에서 정상적인 학습을 할 수 있음




In [ ]:
job_clean.drop(columns="Rating", axis=1, inplace=True)

### ※ 모델링 사전작업
 - 학습 / 검증 / 테스트(20%) 세트 구성

In [ ]:
# train, val, test 분리
train, test = train_test_split(job_clean, test_size=0.2, random_state=21)
train, val = train_test_split(train, test_size=0.2, random_state=21)

train.shape, val.shape, test.shape

In [ ]:
target = "Recommend"
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

### ※ 모델링 (3가지 모델학습)


*   의사결정나무(DesisionTreeClassifier)
*   랜덤포레스트 분류기(RandomForesetClassifier)
*   XGBboost 분류기(XGBClassifier)



In [ ]:
val_result = pd.DataFrame({"모델" : ["의사결정나무", "랜덤포레스트", "XGBoost 분류기"],
                           "정확도" : [0.80, 0.80, 0.87],
                           "f1" : [0.73, 0.74, 0.80],
                           "정밀도" : [0.68, 0.69, 0.83],
                           "재현율" : [0.79, 0.79, 0.78],
                           "Rank" : [3, 2, 1]})

print("※ 3가지 학습 결과(검증데이터)\n ")
val_result = val_result.sort_values(by=["Rank"], axis=0).set_index("Rank")
val_result

In [ ]:
final_result = pd.DataFrame({"구분" : ["훈련", "검증", "테스트"],
                             "정확도" : [0.91, 0.87, 0.84],
                             "정밀도" : ["-", 0.83, 0.81],
                             "재현율" : ["-", 0.78, 0.73],
                             "f1" : [0.86, 0.80, 0.76],
                             "AUC" : ["-", 0.93, 0.91]})

print("※ 최종모델 모델 결과")
final_result.set_index("구분")

## ※ 최종결과


1.   정확도 : 훈련, 검증 결과보다는 하락하였으나, 기준모델 0.649보다 향상되었음
2.   "정밀도 > 재현율" 이 좋은 모델이라 할 수 있음\
   1)   직업에 대해 실제 만족 하면서 비추천 → 정밀도\
   2)   직업에 대해 실제 불만족 하면서 추천 → 재현율
3.    AUC : 0.91 (예측을 얼마나 잘 평가하는지 측정하는 점수, 1에 가까울 수록 좋음)

#### ※ 3가지 모델 학습

In [ ]:
pipe_dtc = make_pipeline(OrdinalEncoder(),
                         DecisionTreeClassifier(random_state=21,
                                                max_depth = 10))

pipe_dtc.fit(X_train, y_train)
y_pred = pipe_dtc.predict(X_val)

print("훈련 정확도 : ", pipe_dtc.score(X_train, y_train))
print("검증 정확도 : ", pipe_dtc.score(X_val, y_val))
print()
print("훈련 f1_score : ", f1_score(pipe_dtc.predict(X_train), y_train))
print("검증 f1_score : ", f1_score(pipe_dtc.predict(X_val), y_val))
print()
print(classification_report(y_val, y_pred))

In [ ]:
pipe_rfc = make_pipeline(OrdinalEncoder(),
                         RandomForestClassifier(max_depth = 7,
                                                n_jobs=-1,
                                                random_state=21,
                                                oob_score=True,
                                                class_weight="balanced"))

pipe_rfc.fit(X_train, y_train)
y_pred = pipe_rfc.predict(X_val)

print("훈련 정확도 : ", pipe_rfc.score(X_train, y_train))
print("검증 정확도 : ", pipe_rfc.score(X_val, y_val))
print()
print("훈련 f1_score : ", f1_score(pipe_rfc.predict(X_train), y_train))
print("검증 f1_score : ", f1_score(pipe_rfc.predict(X_val), y_val))
print()
print(classification_report(y_val, y_pred))

In [ ]:
pipe_xgb = make_pipeline(OrdinalEncoder(),
                         XGBClassifier(max_depth=5,
                                       n_estimators=600,
                                       random_state=21,
                                       learning_rate=0.2,
                                       min_child_weight=100,
                                       n_jobs=-1))

pipe_xgb.fit(X_train, y_train)
y_pred = pipe_xgb.predict(X_val)

print("훈련 정확도 : ", pipe_xgb.score(X_train, y_train))
print("검증 정확도 : ", pipe_xgb.score(X_val, y_val))
print()
print("훈련 f1_score : ", f1_score(pipe_xgb.predict(X_train), y_train))
print("검증 f1_score : ", f1_score(pipe_xgb.predict(X_val), y_val))
print()
print(classification_report(y_val, y_pred))

In [ ]:
# %%time

# pipe = Pipeline([("encoder", OrdinalEncoder()),
#                  ("xgb", XGBClassifier(random_state=21,
#                                       n_jobs=-1))])

# param_grid = {"xgb__n_estimators" : [200, 300, 400, 500, 600, 700, 800, 900, 1000],
#               "xgb__max_depth" : [3,4,5,6,7,8,9,10],
#               "xgb__learning_rate" : [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.5, 0.7, 1.0],
#               "xgb__min_child_weight" : randint(10, 200),
#               "xgb__min_split_loss" : randint(10, 200),
#               "xgb__subsample" : [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#               }

# clf = RandomizedSearchCV(pipe,
#                          param_grid,
#                          n_iter=50,
#                          cv=3,
#                          scoring = "f1",
#                          verbose=1,
#                          n_jobs=-1)

# clf.fit(X_train, y_train)

# print('최적 하이퍼파라미터: ', clf.best_params_)
# print('f1_score: ', clf.best_score_)
# print()
# print('훈련 정확도', clf.score(X_train, y_train))
# print('검증 정확도', clf.score(X_val, y_val))



In [ ]:
k = 3

scores = cross_val_score(pipe_xgb, X_train, y_train, cv=k, scoring="f1")

print("F1_score for {} folds".format(k), scores)

In [ ]:
scores = cross_val_score(pipe_xgb, X_train, y_train, cv=k, scoring="accuracy")

print("Accuracy for {} folds".format(k), scores)

In [ ]:
fig, ax = plt.subplots()
pcm = plot_confusion_matrix(pipe_xgb, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax,
                            values_format="d");
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15)
plt.show()

In [ ]:
cm = pcm.confusion_matrix

total = cm.sum()
correct_prediction = np.diag(cm).sum()

accuracy = correct_prediction / total                # 정확도 : 전체 범주를 모두 바르게 맞춘 경우를 전체 수로 나눈 값
precision = cm[1][1] / (cm[0][1] + cm[1][1])         # 정밀도 : 1로 예측한 경우 올바르게 1을 맞춘 비율
recall = cm[1][1] / (cm[1][0] + cm[1][1])            # 재현율 : 실제 1인 것 중 1로 올바르게 맞춘 비율 
f1 = 2 * (precision * recall) / (precision + recall) # F1점수 : 정밀도와 재현율의 조화평균


print("정확도 : ", accuracy)
print("정밀도 : ", precision)
print("재현율 : ", recall)
print("f1_score : ", f1)

In [ ]:
cm[0][1], cm[1][1]

In [ ]:
y_val_pred_proba = pipe_xgb.predict_proba(X_val)[:,1]

fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_proba)

roc = pd.DataFrame({"FPR" : fpr,
                    "TPR(recall)" : tpr,
                    "Threshold" : thresholds})

plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');

In [ ]:
# threshold 최대값의 인덱스, np.argmax()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print('idx:', optimal_idx, ', threshold:', optimal_threshold)
print("AUC : ", roc_auc_score(y_val, y_val_pred_proba))

y_pred_optimal = y_val_pred_proba >= optimal_threshold
print(classification_report(y_val, y_pred_optimal))

In [ ]:
pipe_xgb = make_pipeline(OrdinalEncoder(),
                         XGBClassifier(max_depth=5,
                                       n_estimators=600,
                                       random_state=21,
                                       learning_rate=0.2,
                                       min_child_weight=100,
                                       n_jobs=-1))

pipe_xgb.fit(X_train, y_train)
y_pred = pipe_xgb.predict(X_test)

print("훈련 정확도 : ", pipe_xgb.score(X_train, y_train))
print("검증 정확도 : ", pipe_xgb.score(X_val, y_val))
print("테스트 정확도 : ", pipe_xgb.score(X_test, y_test))
print()
print("훈련 f1_score : ", f1_score(pipe_xgb.predict(X_train), y_train))
print("검증 f1_score : ", f1_score(pipe_xgb.predict(X_val), y_val))
print("테스트 f1_score : ", f1_score(pipe_xgb.predict(X_test), y_test))
print()
print(classification_report(y_test, y_pred))

In [ ]:
y_test_pred_proba = pipe_xgb.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)

roc = pd.DataFrame({"FPR" : fpr,
                    "TPR(recall)" : tpr,
                    "Threshold" : thresholds})

plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');

In [ ]:
# threshold 최대값의 인덱스, np.argmax()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print('idx:', optimal_idx, ', threshold:', optimal_threshold)
print("AUC : ", roc_auc_score(y_test, y_test_pred_proba))

y_pred_optimal = y_test_pred_proba >= optimal_threshold
print(classification_report(y_test, y_pred_optimal))

# 모델 해석

In [ ]:
# permutation importance
permuter = PermutationImportance(pipe_xgb.named_steps["xgbclassifier"],
                                 scoring="f1",
                                 n_iter=5,
                                 random_state=21)
X_val_encoded = pipe_xgb.named_steps['ordinalencoder'].transform(X_val)
X_test_encoded = pipe_xgb.named_steps['ordinalencoder'].transform(X_test)

permuter.fit(X_val_encoded, y_val)

feature_names = list(X_val.columns)
eli5.show_weights(permuter,
                  top=None,
                  feature_names=feature_names)

In [ ]:
feature = "Size"

isolated = pdp_isolate(pipe_xgb,
                       dataset=X_val_encoded,
                       feature=feature,
                       model_features=X_val.columns,
                       grid_type="percentile",
                       num_grid_points=10)

pdp_plot(isolated, feature_name=feature);

In [ ]:
pdp_plot(isolated
         , feature_name=feature
         , plot_lines=True # ICE plots
         , frac_to_plot=0.01 # or 10 (# 10000 val set * 0.001)
         , plot_pts_dist=True);

In [ ]:
feature = "Revenue"

isolated = pdp_isolate(pipe_xgb,
                       dataset=X_val_encoded,
                       feature=feature,
                       model_features=X_val.columns,
                       grid_type="percentile",
                       num_grid_points=10)

pdp_plot(isolated, feature_name=feature);

In [ ]:
pdp_plot(isolated
         , feature_name=feature
         , plot_lines=True # ICE plots
         , frac_to_plot= 0.01 # or 10 (# 10000 val set * 0.001)
         , plot_pts_dist=True);

In [ ]:
feature = "Average_Salary"

isolated = pdp_isolate(pipe_xgb,
                       dataset=X_val,
                       feature=feature,
                       model_features=X_val.columns,
                       grid_type="percentile",
                       num_grid_points=20)

pdp_plot(isolated, feature_name=feature);

In [ ]:
pdp_plot(isolated
         , feature_name=feature
         , plot_lines=True # ICE plots
         , frac_to_plot= 0.01 # or 10 (# 10000 val set * 0.001)
         , plot_pts_dist=True);

In [ ]:
features_pdp = ["Size", "Average_Salary"] # Average_Salary

interaction = pdp_interact(model=pipe_xgb,
                          dataset=X_val,
                          model_features=X_val.columns,
                          features=features_pdp)

pdp_interact_plot(interaction, plot_type='grid', feature_names=features_pdp);

In [ ]:
# 그렇다면 실제 추천, 비추천 하는 사람들의 데이터 중 어떤 특성들이 큰 영향을 끼치는지 확인해 보겠습니다.

In [ ]:
idx = X_test.index

df_p = pd.DataFrame({"Index" : idx,
                     "pred_proba" : y_test_pred_proba,
                     "status_group" : y_test})

df_p = df_p.merge(job_clean[list(job_clean.columns)], left_index=True, right_index=True)

In [ ]:
df_p

In [ ]:
recommend = df_p['status_group'] == 1
unrecommend = ~recommend
right = recommend == (df_p['pred_proba'] > 0.50)
wrong = ~right

In [ ]:
df_p[recommend & right].sample(n=5, random_state=21).sort_values(by='pred_proba')

In [ ]:
row1 = X_test_encoded.loc[[3560]]
row1

In [ ]:
xgb = pipe_xgb.named_steps["xgbclassifier"]

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(row1)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row1
)

In [ ]:
row2 = X_test_encoded.loc[[427]]
row2

In [ ]:
xgb = pipe_xgb.named_steps["xgbclassifier"]

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(row2)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row2
)

In [ ]:
df_p[unrecommend & right].sample(n=5, random_state=21).sort_values(by='pred_proba')

In [ ]:
row3 = X_test_encoded.loc[[8813]]
row3

In [ ]:
xgb = pipe_xgb.named_steps["xgbclassifier"]

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(row3)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row3
)

In [ ]:
row4 = X_test_encoded.loc[[71]]
row4

In [ ]:
xgb = pipe_xgb.named_steps["xgbclassifier"]

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(row4)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row4
)

In [ ]:
shap_values = explainer.shap_values(X_val_encoded)
shap.summary_plot(shap_values, X_val_encoded, plot_type="violin")

In [ ]:
shap_values = explainer.shap_values(X_test_encoded)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
importance = pd.Series(pipe_xgb.named_steps["xgbclassifier"].feature_importances_, X_val_encoded.columns)

plt.figure(figsize=(10, 10))

importance.sort_values().plot.barh();